In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import scipy as sp
from sklearn.cluster import KMeans
from scipy.spatial import distance_matrix
import os


FILE = "ERCO.csv"
DIRI = "data"
K = 15

file_path = os.path.join(DIRI, FILE)
df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43848 entries, 0 to 43847
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   date_time             43848 non-null  object
 1   raw demand (MW)       43848 non-null  object
 2   category              43848 non-null  object
 3   cleaned demand (MW)   43848 non-null  int64 
 4   forecast demand (MW)  43848 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [2]:
df["date_time"] = pd.to_datetime(df['date_time'], format='%Y-%m-%d %H:%M:%S')
df["date"] = df["date_time"].dt.date
df["hour"] = df["date_time"].dt.hour
df_dropped = df.drop(["raw demand (MW)", "category", "forecast demand (MW)", "date_time"], axis=1)
dfp = df_dropped.pivot(index = "date", columns = "hour", values = "cleaned demand (MW)")
dfp

hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
date,,,,,,,,,,,,,,,,,,,,,
2015-07-02,51756,50171,48655,47667,44629,40754,37547,35199,33637,32759,...,39221,42398,45675,48667,51260,53719,55525,56836,57505,56807
2015-07-03,55044,52776,50986,49845,46817,43307,40173,37954,36353,35346,...,38820,42009,45201,48116,50501,52232,53443,54180,54510,54028
2015-07-04,52732,50673,49028,47940,45385,42470,39566,37379,35757,34698,...,37333,41075,45018,48470,51173,52980,54226,54755,54722,54308
2015-07-05,52974,50742,48474,46782,44767,42872,40437,38209,36459,35240,...,37297,40940,44533,47856,50701,52879,54568,55685,56335,56405
2015-07-06,55512,53679,52057,51238,48381,44766,41716,39612,38216,37343,...,43577,46689,49913,52908,55354,57435,58925,59711,59930,59304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-27,55095,53416,51807,50715,48619,45807,43299,41260,39705,38813,...,41942,45444,48782,51854,54778,57198,58782,60005,60470,60323
2020-06-28,59264,57061,55161,53702,51026,48129,45551,43406,41870,40717,...,42354,45582,48915,51665,53659,55427,56442,57269,57409,57179
2020-06-29,56300,54703,53551,52916,50826,47943,45434,43687,42519,41958,...,47646,50484,53466,56414,59324,61739,63456,64252,64711,64278


In [3]:
dfp.index

Index([2015-07-02, 2015-07-03, 2015-07-04, 2015-07-05, 2015-07-06, 2015-07-07,
       2015-07-08, 2015-07-09, 2015-07-10, 2015-07-11,
       ...
       2020-06-22, 2020-06-23, 2020-06-24, 2020-06-25, 2020-06-26, 2020-06-27,
       2020-06-28, 2020-06-29, 2020-06-30, 2020-07-01],
      dtype='object', name='date', length=1827)

### Create Distance Matrix
Q n x n matrix that containts the distance between each point. The rows will represent "from" and the columns will represent "to" for our MST, but this is arbitrary. 

In [4]:
def create_dist_matrix(X):
   return  pd.DataFrame(distance_matrix(X.values, X.values), index=X.index, columns=X.index)

In [5]:
weights = create_dist_matrix(dfp)
weights

date,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,...,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01
date,,,,,,,,,,,,,,,,,,,,,
2015-07-02,0.000000,10459.280425,7566.910003,7857.485094,19719.162863,21579.584472,16884.717054,15945.260048,19194.485510,15913.139508,...,22276.323844,21410.220643,8653.545401,18463.931488,21090.802261,20196.720353,26863.645024,37714.268918,47373.243545,51608.449076
2015-07-03,10459.280425,0.000000,6145.804423,7395.180390,18316.181562,16675.757644,11015.928150,16597.532618,17716.732402,11994.930262,...,16788.015636,12613.456505,8853.101886,15278.135914,14701.172708,17946.681030,20289.924667,36276.169905,44282.507054,48742.864247
2015-07-04,7566.910003,6145.804423,0.000000,3694.634082,20985.791550,21181.095038,15548.929384,18276.889533,20502.886943,14910.969318,...,21245.889979,18468.236489,10084.255501,18804.236943,19730.434283,20325.149840,24765.081526,38934.968640,47871.664552,52151.606313
2015-07-05,7857.485094,7395.180390,3694.634082,0.000000,20270.230536,21177.538573,15712.892923,17166.963214,19743.944084,14305.598450,...,20388.605690,18597.261465,8779.553405,17915.576686,19459.780934,19027.272453,24176.799292,37756.017163,46778.103831,51054.597658
2015-07-06,19719.162863,18316.181562,20985.791550,20270.230536,0.000000,7795.375039,11839.268981,5152.570621,8356.921562,12616.697627,...,10108.806161,16949.352908,12914.275589,6901.742244,10547.135061,5170.717068,12643.258283,18558.571874,28084.378861,32587.907573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-27,20196.720353,17946.681030,20325.149840,19027.272453,5170.717068,9899.822271,12548.781176,6466.214890,9195.483076,11600.449733,...,9050.099060,16402.924587,12529.643530,6712.120157,10800.471795,0.000000,11111.849306,18951.957814,28095.194625,32509.600782
2020-06-28,26863.645024,20289.924667,24765.081526,24176.799292,12643.258283,8720.142889,13452.578786,16090.500054,14171.241653,14526.315672,...,7564.080513,11970.930290,19125.821812,10904.206528,7948.074735,11111.849306,0.000000,21093.653595,25804.180437,30406.334735
2020-06-29,37714.268918,36276.169905,38934.968640,37756.017163,18558.571874,21924.622505,27944.930453,21919.278775,22837.806812,28756.362583,...,22258.872074,31199.195567,30624.491179,22613.735472,24512.596109,18951.957814,21093.653595,0.000000,13327.896683,17691.321884


In [6]:
mst = sp.sparse.csgraph.minimum_spanning_tree(weights)
mst_arr = mst.toarray().astype(int)
mst_arr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [7]:
np.amax(mst_arr)

19750

In [8]:
mst_arr.sum()

7030904

In [9]:
from numpy import unravel_index

for k in range(K-1):
    x, y = unravel_index(mst_arr.argmax(), mst_arr.shape)
    mst_arr[x][y] = 0

In [10]:
mst_arr.sum()

6825051

In [11]:
# from scipy import sparse
# sA = sparse.csr_matrix(mst_arr)

In [12]:
num_graphs, labels = sp.sparse.csgraph.connected_components(mst_arr, directed=False)
num_graphs

15

In [13]:
results = [[] for i in range(num_graphs + 1)]
for idx, label in enumerate(labels):
    results[label].append(idx)

for idx in range(num_graphs + 1):
    print(len(results[idx]))

1808
1
4
1
3
1
1
1
1
1
1
1
1
1
1
0
